In [2]:
import torch
import os
import sys

sys.path.insert(1, os.path.join(sys.path[0], 'src'))

processed_dir = '/data/pengmiao/PaCKD_0/processed'

train_loader = torch.load(os.path.join(processed_dir, f"bc-3.train.pt"))
test_loader = torch.load(os.path.join(processed_dir, f"bc-3.test.pt"))

In [7]:
from utils import select_tch

device = torch.device(f"cuda:1" if torch.cuda.is_available() else "cpu")

model = select_tch('r')
model = model.to(device)

In [8]:
sigmoid = torch.nn.Sigmoid()
import torch.nn.functional as F

def train(ep, train_loader, model_save_path):
    global steps
    epoch_loss = 0
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):#d,t: (torch.Size([64, 1, 784]),64)        
        optimizer.zero_grad()
        output = sigmoid(model(data))
        loss = F.binary_cross_entropy(output, target, reduction='mean')
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    epoch_loss/=len(train_loader)
    return epoch_loss


def test(test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = sigmoid(model(data))
            test_loss += F.binary_cross_entropy(output, target, reduction='mean').item()
            thresh=0.5
            output_bin=(output>=thresh)*1
            correct+=(output_bin&target.int()).sum()
        test_loss /=  len(test_loader)
        return test_loss

In [9]:
import csv

def run_epoch(epochs, early_stop, loading, model_load_path, model_save_path, train_loader, test_loader, tsv_path, model):
    if loading==True:
        model.load_state_dict(torch.load(model_load_path))
        print("-------------Model Loaded------------")
        
    best_loss=0
    early_stop = early_stop
    curr_early_stop = early_stop

    metrics_data = []

    for epoch in range(epochs):

        train_loss=train(epoch,train_loader,model_save_path)
        test_loss=test(test_loader)
        print((f"Epoch: {epoch+1} - loss: {train_loss:.10f} - test_loss: {test_loss:.10f}"))
        
        if epoch == 0:
            best_loss=test_loss
        if test_loss<=best_loss:
            torch.save(model.state_dict(), model_save_path)    
            best_loss=test_loss
            print("-------- Save Best Model! --------")
            curr_early_stop = early_stop
        else:
            curr_early_stop -= 1
            print("Early Stop Left: {}".format(curr_early_stop))
        if curr_early_stop == 0:
            print("-------- Early Stop! --------")
            break

        metrics_data.append([epoch+1, train_loss, test_loss])

    with open(tsv_path, 'w') as file:
        writer = csv.writer(file, delimiter='\t')
        writer.writerow(['Epoch', 'Train Loss', 'Test Loss'])
        writer.writerows(metrics_data)

Retraining K=2, Past IP, Teacher 2 ResNet -> Teacher No KD No Cluster by introducing entire trace's data

In [10]:
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

from data_loader import init_dataloader

device = torch.device(f"cuda:1" if torch.cuda.is_available() else "cpu")

optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = StepLR(optimizer, step_size=20, gamma=0.1)

epochs = 50
early_stop = 15
loading = True
model_load_path = '/data/pengmiao/PaCKD_2/model/i/bc-3.teacher_2.r.pth'
model_save_path = '/data/pengmiao/PaCKD_1/model/bc-3.teacher.lr.1.r.pth'
tsv_path = '/data/pengmiao/PaCKD_1/model/bc-3.teacher.lr.1.r.tsv'

init_dataloader('1')

run_epoch(epochs, early_stop, loading, model_load_path, model_save_path, train_loader, test_loader, tsv_path, model)

-------------Model Loaded------------
Epoch: 1 - loss: 0.2215119156 - test_loss: 0.2008387401
-------- Save Best Model! --------
Epoch: 2 - loss: 0.2170958445 - test_loss: 0.2085854665
Early Stop Left: 14
Epoch: 3 - loss: 0.2158046641 - test_loss: 0.2338229451
Early Stop Left: 13
Epoch: 4 - loss: 0.2171713014 - test_loss: 0.2151271666
Early Stop Left: 12
Epoch: 5 - loss: 0.2160396559 - test_loss: 0.2090957304
Early Stop Left: 11
Epoch: 6 - loss: 0.2161202540 - test_loss: 0.2329467156
Early Stop Left: 10
Epoch: 7 - loss: 0.2143578019 - test_loss: 0.2059087671
Early Stop Left: 9
Epoch: 8 - loss: 0.2141948035 - test_loss: 0.2270359972
Early Stop Left: 8
Epoch: 9 - loss: 0.2149632967 - test_loss: 0.2280208452
Early Stop Left: 7
Epoch: 10 - loss: 0.2146963712 - test_loss: 0.2191182222
Early Stop Left: 6
Epoch: 11 - loss: 0.2153943658 - test_loss: 0.2153695932
Early Stop Left: 5
Epoch: 12 - loss: 0.2170695223 - test_loss: 0.2265231647
Early Stop Left: 4
Epoch: 13 - loss: 0.2157253964 - test_

In [21]:
from tqdm import tqdm
from numpy import nanargmax, sqrt
from sklearn.metrics import auc, f1_score, precision_score, recall_score, precision_recall_curve, roc_curve
import numpy as np
def threshold_throttleing(test_df,throttle_type="f1",optimal_type="micro",topk=2,threshold=0.5):
    y_score=np.stack(test_df["y_score"])
    y_real=np.stack(test_df["future"])
    best_threshold=0
    if throttle_type=="roc":
        print("throttleing by roc curve")
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        threshold=dict()
        best_threshold_list=[]
        gmeans=dict()
        ix=dict()
        #pdb.set_trace()
        for i in range(BITMAP_SIZE):
            fpr[i], tpr[i], threshold[i] =roc_curve(y_real[:,i],y_score[:,i])
            roc_auc[i] = auc(fpr[i],tpr[i])
            #best:
            gmeans[i] = sqrt(tpr[i]*(1-fpr[i]))
            ix[i]=nanargmax(gmeans[i])
            best_threshold_list.append(threshold[i][ix[i]])
            #print('Dimension: i=%d, Best threshold=%f, G-Mean=%.3f' %(i, threshold[i][ix[i]], gmeans[i][ix[i]]))
        if optimal_type=="indiv":
            best_threshold=best_threshold_list
            y_pred_bin = (y_score-np.array(best_threshold) >0)*1
            test_df["predicted"]= list(y_pred_bin)#(all,[length])
        elif optimal_type=="micro":
            fpr["micro"], tpr["micro"], threshold["micro"] = roc_curve(y_real.ravel(), y_score.ravel())
            roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
            #best:
            gmeans["micro"] = sqrt(tpr["micro"]*(1-fpr["micro"]))
            ix["micro"]=nanargmax(gmeans["micro"])
            best_threshold=threshold["micro"][ix["micro"]]
            print('Best micro threshold=%f, G-Mean=%.3f' %(best_threshold, gmeans["micro"][ix["micro"]]))
            
            y_pred_bin = (y_score-best_threshold >0)*1
            test_df["predicted"]= list(y_pred_bin)#(all,[length])
            
    if throttle_type=="f1":
        print("throttleing by precision-recall curve")
        p = dict()
        r = dict()
        threshold=dict()
        best_threshold_list=[]
        fscore=dict()
        ix=dict()
        
        p["micro"], r["micro"], threshold["micro"]=precision_recall_curve(y_real.ravel(),y_score.ravel())
        fscore["micro"] = (2 * p["micro"] * r["micro"]) / (p["micro"] + r["micro"])
        ix["micro"]=nanargmax(fscore["micro"])
        best_threshold=threshold["micro"][ix["micro"]]
        print('Best micro threshold=%f, fscore=%.3f' %(best_threshold, fscore["micro"][ix["micro"]]))
        y_pred_bin = (y_score-best_threshold >0)*1
        test_df["predicted"]= list(y_pred_bin)
        
    elif throttle_type=="topk":
        print("throttleing by topk:",topk)
        pred_index = torch.tensor(y_score).topk(topk)[1].cpu().detach().numpy()
        y_pred_bin=[to_bitmap(a,BITMAP_SIZE) for a in pred_index]
        test_df["predicted"]= list(y_pred_bin)
        
    elif throttle_type =="fixed_threshold":
        print("throttleing by fixed threshold:",threshold)
        best_threshold=threshold
        y_pred_bin = (y_score-np.array(best_threshold) >0)*1
        test_df["predicted"]= list(y_pred_bin)#(all,[length])
    
    return test_df, best_threshold

def model_prediction(test_loader, test_df, model_save_path):
    print("predicting")
    prediction = []
    model.load_state_dict(torch.load(model_save_path))
    model.to(device)
    model.eval()
    for data, _ in tqdm(test_loader):
        output = sigmoid(model(data))
        prediction.extend(output.cpu().detach().numpy())
    test_df["y_score"] = prediction

    return test_df[['id', 'cycle', 'addr', 'ip', 'block_address', 'future', 'y_score']]

def evaluate(y_test,y_pred_bin):
    f1_score_res=f1_score(y_test, y_pred_bin, average='micro')
    #recall: tp / (tp + fn)
    recall_score_res=recall_score(y_test, y_pred_bin, average='micro')
    #precision: tp / (tp + fp)
    precision_score_res=precision_score(y_test, y_pred_bin, average='micro',zero_division=0)
    print("p,r,f1:",precision_score_res,recall_score_res,f1_score_res)
    return precision_score_res,recall_score_res,f1_score_res

def run_val(test_loader, test_df, app_name, model_save_path):
    res = {}

    print("Validation start")
    test_df = model_prediction(test_loader, test_df, model_save_path)
    df_res, threshold=threshold_throttleing(test_df,throttle_type="f1",optimal_type="micro")
    p,r,f1 = evaluate(np.stack(df_res["future"]), np.stack(df_res["predicted"]))
    res["app"], res["opt_th"], res["p"], res["r"], res["f1"]=[app_name],[threshold],[p],[r],[f1]

    df_res, _ =threshold_throttleing(test_df,throttle_type="fixed_threshold",threshold=0.5)
    p,r,f1 = evaluate(np.stack(df_res["future"]), np.stack(df_res["predicted"]))
    res["p_5"],  res["r_5"], res["f1_5"]=[p],[r],[f1]
    
    return res

In [13]:
test_df = torch.load(os.path.join(processed_dir, 'bc-3.df.pt'))

Validation start
predicting


NameError: name 'tqdm' is not defined

In [22]:
res = run_val(test_loader, test_df, 'bc-3.txt.xz', model_save_path) 

Validation start
predicting


100%|██████████| 1232/1232 [00:25<00:00, 47.51it/s]


throttleing by precision-recall curve
Best micro threshold=0.056499, fscore=0.226


/tmp/ipykernel_758130/2052061821.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df["predicted"]= list(y_pred_bin)


p,r,f1: 0.12727627006169742 0.9965133991113106 0.22572285897364894
throttleing by fixed threshold: 0.5


/tmp/ipykernel_758130/2052061821.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df["predicted"]= list(y_pred_bin)#(all,[length])


p,r,f1: 0.6162575543782114 0.01832335620858573 0.03558854827404823


In [24]:
def run_epoch_no_load(epochs, early_stop, loading, model_save_path, train_loader, test_loader, tsv_path, model):
    if loading==True:
        model.load_state_dict(torch.load(model_save_path))
        print("-------------Model Loaded------------")
        
    best_loss=0
    early_stop = early_stop
    curr_early_stop = early_stop

    metrics_data = []

    for epoch in range(epochs):

        train_loss=train(epoch,train_loader,model_save_path)
        test_loss=test(test_loader)
        print((f"Epoch: {epoch+1} - loss: {train_loss:.10f} - test_loss: {test_loss:.10f}"))
        
        if epoch == 0:
            best_loss=test_loss
        if test_loss<=best_loss:
            torch.save(model.state_dict(), model_save_path)    
            best_loss=test_loss
            print("-------- Save Best Model! --------")
            curr_early_stop = early_stop
        else:
            curr_early_stop -= 1
            print("Early Stop Left: {}".format(curr_early_stop))
        if curr_early_stop == 0:
            print("-------- Early Stop! --------")
            break

        metrics_data.append([epoch+1, train_loss, test_loss])

    with open(tsv_path, 'w') as file:
        writer = csv.writer(file, delimiter='\t')
        writer.writerow(['Epoch', 'Train Loss', 'Test Loss'])
        writer.writerows(metrics_data)

In [25]:
from torch.optim.lr_scheduler import StepLR
import torch.optim as optim

from data_loader import init_dataloader
from utils import select_tch

device = torch.device(f"cuda:3" if torch.cuda.is_available() else "cpu")

model = select_tch('r')
model = model.to(device)

optimizer = optim.Adam(model.parameters(), lr=0.00005)
scheduler = StepLR(optimizer, step_size=20, gamma=0.1)

epochs = 50
early_stop = 15
loading = False
model_save_path = '/data/pengmiao/PaCKD_1/model/bc-3.teacher.lr.05.r.pth'
tsv_path = '/data/pengmiao/PaCKD_1/model/bc-3.teacher.lr.05.r.tsv'

init_dataloader('3')

run_epoch_no_load(epochs, early_stop, loading, model_save_path, train_loader, test_loader, tsv_path, model)

Epoch: 1 - loss: 0.2244955619 - test_loss: 0.2222966755
-------- Save Best Model! --------
Epoch: 2 - loss: 0.2179210031 - test_loss: 0.2229660526
Early Stop Left: 14
Epoch: 3 - loss: 0.2158057630 - test_loss: 0.2224106195
Early Stop Left: 13
Epoch: 4 - loss: 0.2149826372 - test_loss: 0.2238253510
Early Stop Left: 12
Epoch: 5 - loss: 0.2155781214 - test_loss: 0.2249734929
Early Stop Left: 11
Epoch: 6 - loss: 0.2152852979 - test_loss: 0.2204575997
-------- Save Best Model! --------
Epoch: 7 - loss: 0.2180929596 - test_loss: 0.2136492020
-------- Save Best Model! --------
Epoch: 8 - loss: 0.2153300945 - test_loss: 0.2247993299
Early Stop Left: 14
Epoch: 9 - loss: 0.2165446176 - test_loss: 0.2289228892
Early Stop Left: 13
Epoch: 10 - loss: 0.2159636657 - test_loss: 0.2038170932
-------- Save Best Model! --------
Epoch: 11 - loss: 0.2165088190 - test_loss: 0.2206761034
Early Stop Left: 14
Epoch: 12 - loss: 0.2159929908 - test_loss: 0.2188543278
Early Stop Left: 13
Epoch: 13 - loss: 0.21623

In [26]:
res = run_val(test_loader, test_df, 'bc-3.txt.xz', model_save_path)

Validation start
predicting


100%|██████████| 1232/1232 [00:22<00:00, 54.93it/s]


throttleing by precision-recall curve
Best micro threshold=0.157348, fscore=0.258


/tmp/ipykernel_758130/2052061821.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df["predicted"]= list(y_pred_bin)


p,r,f1: 0.15597639930489698 0.7527054404074839 0.25840570197623247
throttleing by fixed threshold: 0.5


/tmp/ipykernel_758130/2052061821.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df["predicted"]= list(y_pred_bin)#(all,[length])


p,r,f1: 0.8383117360588698 0.029330945868631258 0.05667880721763658
